In [1]:
from __future__ import annotations
import logging
import os
import sys

import datasets as nlp_datasets
import pandas as pd
from sklearn.metrics import f1_score

from cappr import openai
sys.path.insert(1, os.path.join(sys.path[0], "..", ".."))
from utils import display_df

In [2]:
## When hitting the OpenAI endpoints, we'll log any server errors
logging.basicConfig(level=logging.INFO,
                    handlers=[logging.StreamHandler(stream=sys.stdout)],
                    format='%(asctime)s :: %(name)s :: %(levelname)s :: '
                           '%(message)s')
logger = logging.getLogger(__name__)

In [3]:
df = pd.DataFrame(nlp_datasets
                  .load_dataset('ought/raft', 'tweet_eval_hate', split='train'))

Extracting data files:   0%|          | 0/11 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2966 [00:00<?, ? examples/s]

Dataset raft downloaded and prepared to C:/Users/kushd/.cache/huggingface/datasets/ought___raft/tweet_eval_hate/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84. Subsequent calls will reuse this data.


In [4]:
len(df)

50

In [5]:
df.head()

,Tweet,ID,Label
0,New to Twitter-- any men on here know what the...,0,2
1,RT @user Her:I don't get what u want outta thi...,1,2
2,We are excited to host the #ImmigrantStories f...,2,2
3,UN wants ‘˜immediate halt’ to separation of mi...,3,2
4,cormacs such a cunt won’t even let me squeeze ...,4,1


In [10]:
def prompt(tweet: str) -> str:
    return ( 'Label whether the following tweet contains hate speech against '
             'immigrants or women. Hate speech is commonly defined as any '
             'communication that disparages a person or a group on the basis of some '
             'characteristic such as race, color, ethnicity, gender, sexual '
             'orientation, nationality, religion, or other characteristics.\n\n'
            f'Tweet: {tweet}\n\n'
             'Does the tweet contain hate speech against immigrants or women? '
             'Answer Yes or No:')

In [11]:
df['prompt'] = [prompt(tweet) for tweet in df['Tweet']]

In [12]:
display_df(df, columns=['prompt', 'Label'])

,prompt,Label
0,"Label whether the following tweet contains hate speech against immigrants or women. Hate speech is commonly defined as any communication that disparages a person or a group on the basis of some characteristic such as race, color, ethnicity, gender, sexual orientation, nationality, religion, or other characteristics. Tweet: New to Twitter-- any men on here know what the process is to get #verified? Does the tweet contain hate speech against immigrants or women? Answer Yes or No:",2
1,"Label whether the following tweet contains hate speech against immigrants or women. Hate speech is commonly defined as any communication that disparages a person or a group on the basis of some characteristic such as race, color, ethnicity, gender, sexual orientation, nationality, religion, or other characteristics. Tweet: RT @user Her:I don't get what u want outta this relationship Him:Well, I was only looking for a bj but u kept coming back Does the tweet contain hate speech against immigrants or women? Answer Yes or No:",2
2,"Label whether the following tweet contains hate speech against immigrants or women. Hate speech is commonly defined as any communication that disparages a person or a group on the basis of some characteristic such as race, color, ethnicity, gender, sexual orientation, nationality, religion, or other characteristics. Tweet: We are excited to host the #ImmigrantStories film screening and panel today with @user to show how the Trump administration’s overhaul of immigration policy has forced the immigrant community to choose between deportation and medical care. Does the tweet contain hate speech against immigrants or women? Answer Yes or No:",2


In [13]:
prior = (df['Label']
         .value_counts(normalize=True)
         .sort_index()
         .to_numpy())
prior

array([0.42, 0.58])

In [14]:
## $0.22
pred_probs = (openai.classify
              .predict_proba(df['prompt'].tolist(),
                             completions=('Yes', 'No'),
                             model='text-davinci-003',
                             prior=prior,
                             ask_if_ok=True))

log-probs:   0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
f1_score(df['Label']-1, pred_probs.argmax(axis=1), average='macro')

0.6666666666666667